Download the [ChatGPT knowledge-graph triplet formatted datasets](https://huggingface.co/datasets/mesolitica/chatgpt-kg-triplets) from Huggingface first if you haven't already. Uncomment and run cell below to download the wikipedia dataset and save it to the current working directory.

In [22]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-kg-triplets/resolve/main/kg-paragraph-wikipedia.translated.jsonl

In [23]:
import json

In [24]:
def rebel_format(triplets):
    """
    Convert
    [['Bruno Santana', 'participant of', '2004 Summer Olympics'],
    ['Bruno Santana', 'participant of', '2008 Summer Olympics'],
    ['Bruno Santana', 'country of citizenship', 'Brazil']]
    to rebel format,
    <triplet> Bruno Santana <subj> 2004 Summer Olympics <obj> participant of <subj> 2008 Summer Olympics <obj> participant of <subj> Brazil <obj> country of citizenship
    """
    q = []
    for no, triple in enumerate(triplets):
        obj = ['<obj>'] + triple[1].split()
        subj = ['<subj>'] + triple[2].split()
        if no > 0 and triple[0] == triplets[no - 1][0]:
            q.extend(subj + obj)
        else:
            triplet = ['<triplet>'] + triple[0].split()
            q.extend(triplet + subj + obj)
    return ' '.join(q)

In [25]:
with open('kg-paragraph-wikipedia.translated.jsonl') as fopen:
    for l in fopen:
        data = json.loads(l)
        if len(data['paragraph_kg_ms']):  
            break

Let's take a look at a sample piece of data before converting it into the rebel format

In [26]:
data['paragraph_kg_ms'][0]

['Menurut bancian India pada tahun 2001, Remuna memiliki bilangan penduduk seramai 28,958. Dari jumlah tersebut, kaum lelaki membentuk 52% dari jumlah keseluruhan populasi dan wanita seramai 48%. ',
 [{'head': 'Remuna', 'type': 'populasi', 'tail': '28958'},
  {'head': 'Remuna', 'type': 'peratusan_populasi lelaki', 'tail': '52'},
  {'head': 'Remuna', 'type': 'peratusan_populasi wanita', 'tail': '48'}]]

In [27]:
with open('processed-kg-wikipedia.jsonl', 'w') as fopen_l:
    with open('kg-paragraph-wikipedia.translated.jsonl') as fopen:
        for l in fopen:
            data = json.loads(l)
            for r in data['paragraph_kg_ms']:
                text, g = r
                if not g:
                    continue
                triples = []
                for t in g:
                    if any([t_ is None for t_ in t.values()]):
                        continue
                    triples.append([t['head'], t['type'], t['tail']])
                kg = rebel_format(triples).replace('_', ' ')

                d = {
                    'text': text.strip(),
                    'kg': kg
                }
                fopen_l.write(f'{json.dumps(d)}\n')
                fopen_l.flush()

In [28]:
# Count the number of lines in the converted dataset
!wc -l processed-kg-wikipedia.jsonl

74318 processed-kg-wikipedia.jsonl


In [29]:
!head -n 3 processed-kg-wikipedia.jsonl

{"text": "Menurut bancian India pada tahun 2001, Remuna memiliki bilangan penduduk seramai 28,958. Dari jumlah tersebut, kaum lelaki membentuk 52% dari jumlah keseluruhan populasi dan wanita seramai 48%.", "kg": "<triplet> Remuna <subj> 28958 <obj> populasi <subj> 52 <obj> peratusan populasi lelaki <subj> 48 <obj> peratusan populasi wanita"}
{"text": "Menurut bancian India pada tahun 2001 Remuna memiliki kadar pendidikan (kebolehan membaca) 61%, melebihi kadar purata kebangsaan 59.5%; dengan 59% lelaki dan 41% wanita mampu membaca. 14% dari populasi berusia di bawah 6 tahun. (2001)", "kg": "<triplet> Remuna <subj> 61% <obj> pendidikan <subj> 59.5% <obj> kadar purata kebangsaan <subj> 59% <obj> lelaki mampu membaca <subj> 41% <obj> wanita mampu membaca <subj> 14% <obj> populasi berumur di bawah 6 tahun"}
{"text": "Pencetak baris tanpa hentaman merupakan sejenis pencetak komputer yang menggunakan kaedah laser, elektrostatik atau kaedah pancutan dakwat adalah pencetak yang terlaju dan pal

In [30]:
!tail -n 3 processed-kg-wikipedia.jsonl

{"text": "Wales dan Beesley kemudian melancarkan sebuah syarikat baru, Wikia, yang tidak mempunyai kaitan dengan Wikimedia ataupun Bomis, walaupun ia mendermakan kepada Wikimedia. Kemudiannya, jawatan-jawatan rasmi yang lain dicipta: Tim Starling dilantik sebagai Pegawai Perhubungan Pembangun Perisian untuk membantu memperbaik pembangunan perisian MediaWiki. Daniel Mayer dilantik sebagai Ketua Pegawai Kewangan untuk membantu menyediakan belanjawan serta menyeleraskan kempen-kempen untuk memperoleh derma penubuhan tabung. Erik M\u00f6ller merupakan Ketua Pegawai Penyelidikan, tetapi beliau meletak jawatan pada Ogos 2005 disebabkan perbezaan pendapat dengan Lembaga. James Forrester kemudian dilantik untuk jawatan itu.", "kg": "<triplet> Wales dan Beesley <subj> syarikat baru <obj> mengasaskan <triplet> syarikat baru <subj> Wikimedia <obj> tidak mempunyai kaitan dengan <subj> Bomis <obj> tidak mempunyai kaitan dengan <subj> Wikimedia <obj> mendermakan kepada <triplet> Tim Starling <subj> 